# CPPM Explicit ions tempering simulation 

### Monte Carlo simulation for protein-protein interactions
Explicit ion concentration is used in the simulation. One protein is placed in origin in the simulation box and the other is placed on a specific distance on the z-axis from origin. For each distance a simulation is performed where a virtual translation move is incorporated to get the force from with the potential of mean force is calculated. However, for the smaller distances, where to two proteins are placed closer to each other, were no or very few accepted moves performed, which resulted in unusable values of the virtual translation force. 

In order to come around this problem, parallel tempering was added to the simulation. It was known that with implicit ions did the previous setup give satisfactory results also for small distances. The tempering boxes were thus built up on a decreasing lambda parameter affecting the kappa and the charge of the ions. In the box representing the full explicit ion case, the lambda is 0. When the kappa is multiplied with the constant it becomes zero and as the charges are multiplied with (1-lambda) they remain unchanged and the first box illustrated the exact explicit ion case. In the other end, the last box will illustrate the implicit ion case. Inbetween are boxes where the ions are less and less "explicit". With the switching inbetween boxes, that the parallel tempering allows, it is desired to be able to measure the potential of mean force also when the proteins are located closer to each other. 

The plotting of the results are done in a separate notebook. 
Note that depending on how you structure your files you might need to make mainor changes to the script, also you need to define your work directory and directory of your faunus installation. 

For this project the two milk-proteins betalac and lactoferrin were studied.

In [ ]:
import pandas as pd
import numpy as np
import os.path
import os
from jinja2 import Template
from itertools import product
import ruamel_yaml as yaml
from io import StringIO 
import json


#change to right testing dir
workdir = '/home/isabel/Documents/SI-betalac-lactoferrin-main/tempering/Dir-Cs-mM'
faunusdir = '/home/isabel/anaconda3/envs/betalac/bin/' # point to Faunus from ~20 December 2022 or later

cs_conc = 0.01 # change to make simulations for other salt conc. (mM)
pH_range = [5.5] # the pH is always set to 5.5 in these simulations
offset = np.linspace(50,70,20)
lambda_val = np.linspace(0,1,11)

dfn = pd.DataFrame(product(lambda_val), columns=[ 'la'])
dfn.T

### Create directories for each offset

In directory of salt concentration, create directories for each offset to make overview of data and plotting later easier.

In [ ]:
%cd -q $workdir
for off in offset:
    dirname = f'Off{off}' 
    if not os.path.isdir(dirname):
        print(dirname)
        os.makedirs(dirname)

### Create input for simulations

Making the input files with function below. For the explicit ion simulations grand canonical salt was used which then adjusted number of salt particles to fit with the concentration and protein charges. 
For each protein-protein mass center distance, here called offset, one mpi for each lambda value are created and put in respective offset directory. 

In [8]:
#tempering input

def makeFaunusInput(outfile, **kwargs):
    ''' generate faunus input file from Jinja2 template '''
    with open('../../temper-template.yml', 'r') as file: # this is the template file
        template = Template(file.read())
        mpi_rank = kwargs['mpi']
        with open(f'mpi{mpi_rank}.' + outfile, 'w') as f:   # open a new faunus input file
            inputstr = template.render(kwargs)
            d = yaml.safe_load(StringIO(inputstr)) # convert string to stream
            f.write(json.dumps(d))
            
for off in offset:
    for idx, row in dfn.iterrows():
        %cd -q $workdir/Off$off
        makeFaunusInput('input.json', Cs=cs_conc, pH=5.5, mpi=idx, off_val=off, micro=1000, la= row.la)         

### Run simulations

Simulations for respective offset are run in parallel with mpi such that parallel tempering can be performed. These simulations were tested on a local computer and later performed on a cluster in order to aloow long enough simulation times to reach reliable results.  

In [ ]:
for off in offset:
    %cd -q $workdir/Off$off
    !pwd
    !mpirun -np 11 /home/isabel/github/faunus/faunus -i input.json -o out.json -v 6 2> log #-s state.json